<b>Install required dependencies and libraries</b>

In [1]:
! pip install lxml html5lib beautifulsoup4

In [2]:
import pandas as pd
import numpy as np

<b>Extract Data Table from URL</b>

In [3]:
#Webpage URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Extract Tables
dfs = pd.read_html(url)

#Get the first table
df=dfs[0]

#Extract Columns
dfYTO = df[['Postal Code','Borough', 'Neighbourhood']]
print(dfYTO)        

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

In [4]:
#Check initial entries:
dfYTO.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#Drop the Borough Rows showing 'Not Assigned'  and save as a new data frame
dfYTO1 = dfYTO[dfYTO.Borough != 'Not assigned']

In [10]:
dfYTO1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<b>I have used the duplicated function to find out if there was any duplicated Postal Code in the dataframe:</b>

In [11]:
#Command to generate a Dupe Dataframe only for the column Postal Code
PCdupe=dfYTO1['Postal Code'].duplicated()

In [12]:
#Check the PCdupe dataframe: 
PCdupe.head()

2    False
3    False
4    False
5    False
6    False
Name: Postal Code, dtype: bool

In [13]:
#I will also do a value count, only values showing as True would be duplicated: 
PCdupe.value_counts()

False    103
Name: Postal Code, dtype: int64

My finding after the counting and the duplicated function, indicates that there are not duplicated postal codes on the main dataframe dfYTO1. 

<b>Now, I will look for any <i>Not assigned</i> variables in the Neighbourhood column.</b>

In [14]:
#Use .loc function look up for any not assigned variables: 
dfnhood=dfYTO1.loc[dfYTO1['Neighbourhood']=='Not assigned']

In [15]:
dfnhood

,Postal Code,Borough,Neighbourhood


As per the above there are no cells showing as Not Assigned in the current dfYTO1 dataframe.

In [17]:
#shows the shape of my dataframe as it stands now: 
dfYTO1.shape

(103, 3)

After the initial data processing. I have 103 rows and 3 columns.